# usual imports

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

# Get fruits and vegetables names and urls

## First page of vegetable names

In [2]:
html = urlopen("https://en.wikipedia.org/wiki/List_of_vegetables") 
res = BeautifulSoup(html.read(),"html5lib")
all_urls = sum([[ c.find("a")['href'] for c in res.findAll("table")[i].findAll("tr")][1:] for i in range(7)], [])
all_names = sum([[ c.find("a").text for c in res.findAll("table")[i].findAll("tr")][1:] for i in range(7)], [])

## Second page of vegetable names

In [3]:
html = urlopen("https://en.wikipedia.org/wiki/List_of_culinary_fruits") 
res = BeautifulSoup(html.read(),"html5lib")

In [4]:
#names = 
true_names = []
true_urls = []

candidates = [c.findAll("li") for c in res.findAll("div", {"class": "div-col columns column-width"})][:24]
    
for candidate in candidates :
    
    for b in candidate :   
        
        info = b.find("a")
        
        if info is not None :
            #print(b)
            true_names.append(info.text)
            true_urls.append(info["href"])
    
    
#len(true_names)
#len(true_urls)

all_names.extend(true_names)
all_urls.extend(true_urls)

# Amount parsing

In [5]:
def parse_amount(name, amount) :
    
    try :
    
        if 'g' not in amount :
            if 'IU' not in amount :
                return 0
            else :

                if '%' in amount :            
                    #get rid of %
                    if amount.index('%') < amount.index('U') :
                        amount = amount[amount.index('%')+1:amount.index('U')]

                    else :
                        amount = amount[:amount.index('U')]

                #get rid of UI
                amount = float(re.sub('[UI]','',amount).strip())

                #see https://en.wikipedia.org/wiki/International_unit#Mass_equivalents_of_1_IU
                if name == "Vitamin A" :
                    amount = amount * 0.0006

                elif name == "Vitamin C" :
                    amount = amount * 0.05

                elif name == "Vitamin D" :
                    amount = (amount / 40) * 0.001

                elif name == "Vitamin E" :
                    amount = amount * 2/3

                else : 
                    print("unknown value for ", amount , "and name ", name)
                    return -1


        elif '%' not in amount :

            amount = amount[:amount.index('g')]

            if len(amount.replace("µ", '')) != len(amount) or  len(re.sub('[μ]','',amount)) != len(amount):

                amount = re.sub('[μ]','',amount)
                amount = amount.replace("µ", '')
                return str(float(re.sub('[mgμ]','',amount).strip())/1000.0)

            elif 'm' not in amount :
                return str(float(re.sub('[mgμ]','',amount).strip())*1000.0)

            else :
                return re.sub('[mμg]','',amount).strip()


        else :

            to_parse = ""

            #determine if % is before or after g
            if amount.index('%') < amount.index('g') :
                to_parse = amount[amount.index('%')+1:amount.index('g')]

            else :
                to_parse = amount[:amount.index('g')]

            if len(to_parse.replace("µ", '')) != len(to_parse) or len(re.sub('[μ]','',to_parse)) != len(to_parse):

                to_parse = to_parse.replace("µ", '')
                to_parse = re.sub('[μ]','',to_parse)
                return str(float(re.sub('[mgμ]','',to_parse).strip())/1000.0)

            elif 'm' not in to_parse :
                return str(float(re.sub('[mgμ]','',to_parse).strip())*1000.0)

            else :
                return re.sub('[mgμ]','',to_parse).strip()
    except :
        return -1
        

# load wanted elements names

In [6]:
elements_names = pd.read_excel('../data/RDI.xlsx')['element'].values
elements_names

array(['alpha-linoleic acid', 'Biotin', 'Calcium', 'Carbohydrate',
       'Chloride', 'Choline', 'Chromium', 'Copper', 'Fat', 'Fiber',
       'Fluoride', 'Folate', 'Iodine', 'Iron', 'Linoleic acid',
       'Magnesium', 'Manganese', 'Molybdenum', 'Monousaturated fat',
       'Niacin', 'Pantothenic acid', 'Phosphorus', 'Polyunsaturated fat',
       'Potassium', 'Protein', 'Riboflavin', 'Saturated fat', 'Selenium',
       'Sodium', 'Sugar', 'Thiamin', 'Vitamin A', 'Vitamin B6',
       'Vitamin B12', 'Vitamin C', 'Vitamin D', 'Vitamin E', 'Vitamin K',
       'Water', 'Zinc'], dtype=object)

# look up the wikipedia tables

In [7]:
def nutritional_facts(url) :
    try : 
        html = urlopen(url) 
        res = BeautifulSoup(html.read(),"html5lib");
        
        food = res.find("h1").text

        nutritional_facts = {}

        table = res.find("table", {"class": "infobox nowrap"}) 
        table_body = table.find('tbody')


        rows = table_body.find_all('tr')
    except :
        print("No information for ", url, '\n')
        return None 
       
    
    for row in rows:

        nut_element = ""

        valid_name = False
        found_name = ""
        
        try :
        
            #find elements names
            elem = row.find('th') 

            if elem is not None :
                name = elem.find('a') 

                if name is None and elem.text is not None :
                    name = elem


                if name is not None :

                    #check if the name contains something interesting
                    matching = [c.lower() in name.text.lower() for c in elements_names]
                    valid_name  = sum(matching) >= 1            

                    if valid_name :
                        matching_name = elements_names[matching.index(True)]
                        found_name = name.text
                        nut_element = matching_name

        except :
            valid_name = False
                        
        #find element amount


        if valid_name :
            
            amount = row.find('td')

            if amount is not None :  

                #hard special case
                if found_name == "Vitamin A equiv." :
                    whole_string = amount.text
                    div_strings = [amount.find_all('div')[i].text for i in range(len(amount.find_all('div')))]
                    div_strings = sorted(div_strings, key = len, reverse = True)

                    #print("whole string : ", whole_string)
                    #print("div_strings : ", div_strings)

                    original_string = whole_string

                    for div_string in div_strings :
                        whole_string = whole_string.replace(div_string, '')

                    if len(whole_string) != 0 :
                        #print("sent to parsing : ", whole_string)
                        nutritional_facts[matching_name] = parse_amount(nut_element, whole_string)

                    else :
                        nutritional_facts[matching_name] = sum([float(parse_amount(nut_element, d)) for d in div_strings])

                elif 'div' not in str(amount) :

                    #print("sent to parsing : ", amount.text)
                    nutritional_facts[matching_name] = parse_amount(nut_element, amount.text)

                else :              

                    #amount = parse_amount(amount.find_all('div')[0].text)
                    #print("sent to parsing : ", amount.text)
                    amount = parse_amount(nut_element, amount.text)
                    nutritional_facts[matching_name] = amount

    return nutritional_facts

In [8]:
dataframe = pd.DataFrame(columns = np.append(elements_names, 'name'))
row = -1
name_index = -1
for url in all_urls :
    name_index = name_index + 1
    print("analyzing ", url, "...")
    nv = nutritional_facts("https://en.wikipedia.org/" + url)
    
    if nv is not None :
        row = row + 1
        nv['name'] = all_names[name_index]
        
        for k in nv.keys() :            
            dataframe.loc[row,k] = nv[k]
        
print("percentage of parsed information : ", 100 * len(dataframe.index) / len(all_urls))      
dataframe.set_index("name")
    

analyzing  /wiki/Amaranth ...
analyzing  /wiki/Arugula ...
No information for  https://en.wikipedia.org//wiki/Arugula 

analyzing  /wiki/Beet ...
analyzing  /wiki/Chinese_cabbage ...
No information for  https://en.wikipedia.org//wiki/Chinese_cabbage 

analyzing  /wiki/Borage ...
No information for  https://en.wikipedia.org//wiki/Borage 

analyzing  /wiki/Broccoli ...
analyzing  /wiki/Brussels_sprout ...
analyzing  /wiki/Cabbage ...
analyzing  /wiki/Hypochaeris_radicata ...
No information for  https://en.wikipedia.org//wiki/Hypochaeris_radicata 

analyzing  /wiki/Celery ...
analyzing  /wiki/Celtuce ...
analyzing  /wiki/Chaya_(plant) ...
No information for  https://en.wikipedia.org//wiki/Chaya_(plant) 

analyzing  /wiki/Stellaria ...
No information for  https://en.wikipedia.org//wiki/Stellaria 

analyzing  /wiki/Chicory ...
analyzing  /wiki/Chinese_mallow ...
No information for  https://en.wikipedia.org//wiki/Chinese_mallow 

analyzing  /wiki/Garland_Chrysanthemum ...
analyzing  /wiki/Co

analyzing  /wiki/Lupinus_mutabilis ...
No information for  https://en.wikipedia.org//wiki/Lupinus_mutabilis 

analyzing  /wiki/Tepary_bean ...
No information for  https://en.wikipedia.org//wiki/Tepary_bean 

analyzing  /wiki/Urad_(bean) ...
analyzing  /wiki/Mucuna_pruriens ...
No information for  https://en.wikipedia.org//wiki/Mucuna_pruriens 

analyzing  /wiki/Winged_bean ...
analyzing  /wiki/Yardlong_bean ...
analyzing  /wiki/Asparagus ...
analyzing  /wiki/Cardoon ...
analyzing  /wiki/Celeriac ...
analyzing  /wiki/Celery ...
analyzing  /wiki/Chives ...
analyzing  /wiki/Elephant_garlic ...
No information for  https://en.wikipedia.org//wiki/Elephant_garlic 

analyzing  /wiki/Fennel ...
analyzing  /wiki/Garlic ...
analyzing  /wiki/Allium_tuberosum ...
No information for  https://en.wikipedia.org//wiki/Allium_tuberosum 

analyzing  /wiki/Kohlrabi ...
analyzing  /wiki/Kurrat ...
No information for  https://en.wikipedia.org//wiki/Kurrat 

analyzing  /wiki/Lemongrass ...
No information for 

No information for  https://en.wikipedia.org//wiki/Kei_apple 

analyzing  /wiki/Mammee_apple ...
No information for  https://en.wikipedia.org//wiki/Mammee_apple 

analyzing  /wiki/Apricot ...
analyzing  /wiki/Araza ...
No information for  https://en.wikipedia.org//wiki/Araza 

analyzing  /wiki/Siraitia_grosvenorii ...
No information for  https://en.wikipedia.org//wiki/Siraitia_grosvenorii 

analyzing  /wiki/Atemoya ...
No information for  https://en.wikipedia.org//wiki/Atemoya 

analyzing  /wiki/Rubus_probus ...
No information for  https://en.wikipedia.org//wiki/Rubus_probus 

analyzing  /wiki/Avocado ...
analyzing  /wiki/Babaco ...
No information for  https://en.wikipedia.org//wiki/Babaco 

analyzing  /wiki/Bacuri ...
No information for  https://en.wikipedia.org//wiki/Bacuri 

analyzing  /wiki/Bael ...
No information for  https://en.wikipedia.org//wiki/Bael 

analyzing  /wiki/Banana ...
analyzing  /wiki/Passiflora_ligularis ...
No information for  https://en.wikipedia.org//wiki/Passif

No information for  https://en.wikipedia.org//wiki/Ficus_racemosa 

analyzing  /wiki/Planchonia_careya ...
No information for  https://en.wikipedia.org//wiki/Planchonia_careya 

analyzing  /wiki/Coco_plum ...
No information for  https://en.wikipedia.org//wiki/Coco_plum 

analyzing  /wiki/Cocona ...
No information for  https://en.wikipedia.org//wiki/Cocona 

analyzing  /wiki/Coconut ...
analyzing  /wiki/Cocoplum ...
No information for  https://en.wikipedia.org//wiki/Cocoplum 

analyzing  /wiki/Coffea ...
No information for  https://en.wikipedia.org//wiki/Coffea 

analyzing  /wiki/Cola_nut ...
No information for  https://en.wikipedia.org//wiki/Cola_nut 

analyzing  /wiki/Billardiera_scandens ...
No information for  https://en.wikipedia.org//wiki/Billardiera_scandens 

analyzing  /wiki/Carissa_lanceolata ...
No information for  https://en.wikipedia.org//wiki/Carissa_lanceolata 

analyzing  /wiki/Cornelian_cherry ...
No information for  https://en.wikipedia.org//wiki/Cornelian_cherry 

ana

No information for  https://en.wikipedia.org//wiki/Indian_almond 

analyzing  /wiki/Indian_gooseberry ...
No information for  https://en.wikipedia.org//wiki/Indian_gooseberry 

analyzing  /wiki/Indian_jujube ...
analyzing  /wiki/Indian_prune ...
No information for  https://en.wikipedia.org//wiki/Indian_prune 

analyzing  /wiki/Jabuticaba ...
No information for  https://en.wikipedia.org//wiki/Jabuticaba 

analyzing  /wiki/Jackfruit ...
analyzing  /wiki/Muntingia ...
No information for  https://en.wikipedia.org//wiki/Muntingia 

analyzing  /wiki/Jambul ...
analyzing  /wiki/Japanese_persimmon ...
No information for  https://en.wikipedia.org//wiki/Japanese_persimmon 

analyzing  /wiki/Hymenaea_courbaril ...
No information for  https://en.wikipedia.org//wiki/Hymenaea_courbaril 

analyzing  /wiki/Jelly_palm ...
No information for  https://en.wikipedia.org//wiki/Jelly_palm 

analyzing  /wiki/Jocote ...
No information for  https://en.wikipedia.org//wiki/Jocote 

analyzing  /wiki/Jujube ...
ana

No information for  https://en.wikipedia.org//w/index.php?title=Monkey_jackfruit&action=edit&redlink=1 

analyzing  /w/index.php?title=Monkey_tamarind&action=edit&redlink=1 ...
No information for  https://en.wikipedia.org//w/index.php?title=Monkey_tamarind&action=edit&redlink=1 

analyzing  /wiki/Monstera ...
No information for  https://en.wikipedia.org//wiki/Monstera 

analyzing  /w/index.php?title=Montessa_granadilla&action=edit&redlink=1 ...
No information for  https://en.wikipedia.org//w/index.php?title=Montessa_granadilla&action=edit&redlink=1 

analyzing  /wiki/Rubus_adenotrichus ...
No information for  https://en.wikipedia.org//wiki/Rubus_adenotrichus 

analyzing  /wiki/Mora_de_Castilla ...
No information for  https://en.wikipedia.org//wiki/Mora_de_Castilla 

analyzing  /wiki/Morinda ...
No information for  https://en.wikipedia.org//wiki/Morinda 

analyzing  /wiki/Morinda_citrifolia ...
No information for  https://en.wikipedia.org//wiki/Morinda_citrifolia 

analyzing  /wiki/Mort

No information for  https://en.wikipedia.org//wiki/Rubus_rosifolius 

analyzing  /wiki/Archirhodomyrtus_beckleri ...
No information for  https://en.wikipedia.org//wiki/Archirhodomyrtus_beckleri 

analyzing  /wiki/Rough_shell_macadamia ...
No information for  https://en.wikipedia.org//wiki/Rough_shell_macadamia 

analyzing  /wiki/Rowan ...
No information for  https://en.wikipedia.org//wiki/Rowan 

analyzing  /wiki/Safou ...
No information for  https://en.wikipedia.org//wiki/Safou 

analyzing  /wiki/Sageretia_theezans ...
No information for  https://en.wikipedia.org//wiki/Sageretia_theezans 

analyzing  /wiki/Saguaro ...
No information for  https://en.wikipedia.org//wiki/Saguaro 

analyzing  /wiki/Salak ...
analyzing  /wiki/Salal ...
No information for  https://en.wikipedia.org//wiki/Salal 

analyzing  /wiki/Salmonberry ...
No information for  https://en.wikipedia.org//wiki/Salmonberry 

analyzing  /wiki/Ficus_coronata ...
No information for  https://en.wikipedia.org//wiki/Ficus_coronata

,alpha-linoleic acid,Biotin,Calcium,Carbohydrate,Chloride,Choline,Chromium,Copper,Fat,Fiber,...,Thiamin,Vitamin A,Vitamin B6,Vitamin B12,Vitamin C,Vitamin D,Vitamin E,Vitamin K,Water,Zinc
name,,,,,,,,,,,,,,,,,,,,,
Amaranth,NaN,NaN,159,65250.0,NaN,NaN,NaN,NaN,7020.0,6700.0,...,0.116,NaN,0.591,NaN,4.2,NaN,1.19,NaN,11130.0,2.87
Beet,NaN,NaN,16,9560.0,NaN,NaN,NaN,NaN,170.0,2800.0,...,0.031,0.002,0.067,NaN,4.9,NaN,NaN,NaN,87580.0,0.35
Broccoli,NaN,NaN,47,6640.0,NaN,NaN,NaN,NaN,370.0,2600.0,...,0.071,0.031,0.175,NaN,89.2,NaN,0.78,0.1016,89300.0,0.41
Brussels sprouts,NaN,NaN,42,8950.0,NaN,19.1,NaN,NaN,300.0,NaN,...,0.139,0.038,0.219,NaN,85,NaN,0.88,0.177,86000.0,0.42
Cabbage,NaN,NaN,40,5800.0,NaN,NaN,NaN,NaN,100.0,2500.0,...,0.061,NaN,0.124,NaN,36.6,NaN,NaN,0.076,NaN,0.18
Celery,NaN,NaN,40,2970.0,NaN,6.1,NaN,0.035,170.0,1600.0,...,0.021,0.022,0.074,0.0,3.1,None,0.27,0.0293,95430.0,0.13
Celtuce,NaN,NaN,39,3650.0,NaN,NaN,NaN,NaN,300.0,1700.0,...,0.055,0.175,0.05,NaN,19.5,NaN,NaN,NaN,NaN,0.27
Chicory,NaN,NaN,100,4700.0,NaN,NaN,NaN,NaN,300.0,4000.0,...,0.06,0.286,0.105,NaN,24,NaN,2.26,0.29760000000000003,NaN,0.42
Garland Chrysanthemum,NaN,NaN,117,3020.0,NaN,NaN,NaN,NaN,560.0,3000.0,...,0.13,0.116,0.176,NaN,1.4,NaN,NaN,0.35,NaN,0.71
